## Soccer predict notepad
Models for predict results soccer games
### Source date:
Ranking - https://www.fifa.com/fifa-world-ranking/ranking-table/men/ <br/>
Results of international championship and selecting games

In [2]:
import pandas as pd
import numpy as np
import re

In [50]:
df_rank=pd.read_csv('ranking.csv', header=0, sep=';')

df_rank['Country Code']=df_rank['Team'].str.extract('^([A-Z]{3})')
df_rank['Team']=df_rank['Team'].apply(lambda x: re.sub(r'^[A-Z]{3}', '', x))
df_rank['Total Points']=df_rank['Total Points'].str.extract('\d+\D(\d+\.\d+)').astype(float)
df_rank.dropna(axis=1, how='all', inplace=True)
df_rank.set_index('Country Code', inplace=True)

df_rank # rank of national team

/home/egor/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  app.launch_new_instance()
/home/egor/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:5: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


,Rank,Team,Total Points,Previous Points,+/-,Avg.,AVG WGT,Avg..1,AVG WGT.1,Avg..2,AVG WGT.2,Avg..3,AVG WGT.3
Country Code,,,,,,,,,,,,,
BRA,1,Brazil,1715.02,1672,0,1038.91,1038.91,555.56,277.78,793.26,237.98,801.78,160.36
ARG,2,Argentina,1626.23,1603,0,877.12,877.12,741.10,370.55,919.87,275.96,512.98,102.60
GER,3,Germany,1511.44,1464,0,857.80,857.80,412.01,206.00,1153.12,345.94,508.50,101.70
CHI,4,Chile,1422.14,1411,0,756.47,756.47,793.85,396.92,489.13,146.74,610.01,122.00
COL,5,Colombia,1365.63,1348,0,705.66,705.66,597.24,298.62,821.59,246.48,574.37,114.87
FRA,6,France,1332.16,1294,0,855.30,855.30,351.26,175.63,704.66,211.40,449.18,89.84
BEL,7,Belgium,1291.57,1281,0,597.83,597.83,587.90,293.95,961.05,288.32,557.40,111.48
POR,8,Portugal,1266.76,1259,0,671.59,671.59,564.84,282.42,617.62,185.29,637.29,127.46
SUI,9,Switzerland,1263.29,1212,0,788.68,788.68,355.32,177.66,563.16,168.95,640.01,128.00


In [51]:
df_rus_eng_translate=pd.read_csv('../Migration/countries_rus_eng.csv', header=None, names=['Rus', 'Eng'])
#df_rus_eng_translate.set_index('Rus', inplace=True)

dict_translate=df_rus_eng_translate.set_index('Rus').to_dict()['Eng']
print(dict_translate)

{'Абхазия': 'Abkhazia', 'Австралия': 'Australia', 'Австрия': 'Austria', 'Азербайджан': 'Azerbaijan', 'Аландские острова': 'Aland Islands', 'Албания': 'Albania', 'Алжир': 'Algeria', 'Ангилья': 'Anguilla', 'Англия': 'England', 'Ангола': 'Angola', 'Андорра': 'Andorra', 'Аргентина': 'Argentina', 'Армения': 'Armenia', 'Аруба': 'Aruba', 'Афганистан': 'Afghanistan', 'Багамские острова': 'Bahamas', 'Бангладеш': 'Bangladesh', 'Барбадос': 'Barbados', 'Бахрейн': 'Bahrain', 'Беларусь': 'Belarus', 'Белиз': 'Belize', 'Бельгия': 'Belgium', 'Бенин': 'Benin', 'Болгария': 'Bulgaria', 'Боливия': 'Bolivia', 'Боливия, многонациональное государство': 'Bolivia', 'Босния и Герцеговина': 'Bosnia and Herzegovina', 'Ботсвана': 'Botswana', 'Бразилия': 'Brazil', 'Бруней': 'Brunei Darussalam', 'Бурунди': 'Burundi', 'Бутан': 'Bhutan', 'Ватикан': 'Vatican City', 'Великобритания': 'United Kingdom', 'Великобритания (Соединенное Королевство)': 'United Kingdom', 'Венгрия': 'Hungary', 'Венесуэла': 'Venezuela', 'Венесуэла,

## Making dataframe for modeling
### Field 'Result' will be y - label for forecasting. 
'1' - left team win, '-1' - right team win, '0' - draw
### Modeling will be on UEFA rank 
Fields 'Total Points', 'Previous Points' and '+/-' from dataset

In [5]:
lst_columns=['Country 1', 'Country 2', 'Score 1', 'Score 2']
df_results=pd.DataFrame(columns=lst_columns)

def set_results(x):
    if x['Score 1'] > x['Score 2']: return 1
    if x['Score 1'] < x['Score 2']: return -1
    return 0

def translate_line(strLine):
    words=list(filter(None, re.split('\t| - |:|–', line))) 
    return [dict_translate.get(word.strip(), word.strip()) for word in words if word not in ['-', '–']]

    
f=open('results.txt', 'r')
for i, line in enumerate(f):
    df_results.loc[i]=dict(zip(lst_columns, translate_line(line)))

df_results[lst_columns[2:]].astype(int)

#print(df_results['Score 2'].sort_values().unique().tolist())
df_results['result']=df_results.apply(set_results, axis=1)
df_results # dataframe with games results

,Country 1,Country 2,Score 1,Score 2,result
0,Brazil,Spain,3,0,1
1,Uruguay,Italy,2,3,-1
2,Spain,Italy,7,6,1
3,Brazil,Uruguay,2,1,1
4,Italy,Brazil,2,4,-1
5,Japan,Mexico,1,2,-1
6,Italy,Japan,4,3,1
7,Brazil,Mexico,2,0,1
8,Mexico,Italy,1,2,-1
9,Brazil,Japan,3,0,1


In [52]:
lstR=pd.DataFrame(df_rank['Team'].sort_values().unique(), columns=['Country'])
ser_code=lstR.reset_index().set_index('Country')['index']

df_rank['Country ID']=df_rank['Team'].map(ser_code)
df_results['Country 1 ID']=df_results['Country 1'].map(ser_code)
df_results['Country 2 ID']=df_results['Country 2'].map(ser_code)

pd_model=pd.merge(df_results, df_rank.dropna(), how='inner', left_on='Country 1 ID', right_on='Country ID')
pd_model=pd.merge(pd_model, df_rank.dropna(), how='inner', left_on='Country 2 ID', right_on='Country ID')
print(pd_model)

            Country 1     Country 2 Score 1 Score 2  result  Country 1 ID  \
0              Brazil         Spain       3       0       1          27.0   
1               Italy         Spain       2       0       1          95.0   
2             Nigeria         Spain       0       3      -1         139.0   
3         New Zealand         Spain       0       5      -1         136.0   
4             Croatia         Spain       2       1       1          50.0   
5       FYR Macedonia         Spain       1       2      -1          67.0   
6              Brazil       Uruguay       2       1       1          27.0   
7               Spain       Uruguay       2       1       1         173.0   
8               Italy       Uruguay       0       1      -1          95.0   
9              Mexico       Uruguay       3       1       1         124.0   
10            Nigeria       Uruguay       1       2      -1         139.0   
11           Colombia       Uruguay       2       0       1          43.0   

In [7]:
lst_model_fld=['result', 'Total Points_x', 
               'Previous Points_x', '+/-_x', 
               'Total Points_y', 
               'Previous Points_y', '+/-_y']
print(pd_model[lst_model_fld]) # dataframe for model

# label = result, 1 - win Country 1, -1 - win Country 2, 0 - draw

     result  Total Points_x  Previous Points_x  +/-_x  Total Points_y  \
0         1         1715.02               1672      0         1198.19   
1         1         1193.23               1165      0         1198.19   
2        -1          729.70                726      2         1198.19   
3        -1          343.62                301     17         1198.19   
4         1         1033.11               1016      0         1198.19   
5        -1          217.80                225     -3         1198.19   
6         1         1715.02               1672      0         1098.77   
7         1         1198.19               1204      0         1098.77   
8        -1         1193.23               1165      0         1098.77   
9         1         1049.98               1076     -1         1098.77   
10       -1          729.70                726      2         1098.77   
11        1         1365.63               1348      0         1098.77   
12        1         1715.02               1672     

In [53]:
#'Brazil', 'Germania'
#print(ser_code.index.tolist())

def make_test_data(strCountry1Name, strCountry2Name):
    sc1=ser_code[strCountry1Name]
    sc2=ser_code[strCountry2Name]
    pd1=df_rank[df_rank['Team']==strCountry1Name][['Total Points', 
               'Previous Points', '+/-', ]]
    pd2=df_rank[df_rank['Team']==strCountry2Name][['Total Points', 
               'Previous Points', '+/-', ]]
    return { 'Total Points_x':pd1['Total Points'].values[0], 
             'Previous Points_x':pd1['Previous Points'].values[0],  '+/-_x':pd1['+/-'].values[0],  
             'Total Points_y':pd2['Total Points'].values[0], 
             'Previous Points_y':pd2['Previous Points'].values[0], '+/-_y':pd1['+/-'].values[0]}

print(make_test_data('Brazil', 'Germany'))

{'Total Points_x': 1715.02, 'Previous Points_x': 1672, '+/-_x': 0, 'Total Points_y': 1511.4400000000001, 'Previous Points_y': 1464, '+/-_y': 0}


## Let's do modeling
Testing models on 2017 FIFA Confederations Cup

In [54]:
from sklearn.model_selection import train_test_split

X=pd_model[lst_model_fld[1:]]
y=pd_model[lst_model_fld[0]]

X_train, X_test, y_train, y_test=train_test_split(X, y, random_state=0)


In [10]:
interst= [['Россия', 'Португалия'],
['Мексика', 'Новая Зеландия'],
['Мексика', 'Россия'],
['Новая Зеландия', 'Португалия'],
['Австралия', 'Германия'],
['Камерун', 'Австралия'],
['Германия', 'Чили'],
['Германия', 'Камерун'],
['Чили', 'Австралия']]

query_list=[]
for_result_lst=[]

for item in interst:
    itm=[dict_translate.get(word.strip(), word.strip()) for word in item]
    for_result_lst.append(['-'.join(itm)])
    #print(itm)
    dct=make_test_data(itm[0], itm[1])
    query_list.append(dct)
pd_query=pd.DataFrame(query_list)
print(for_result_lst)
pd_query

[['Russia-Portugal'], ['Mexico-New Zealand'], ['Mexico-Russia'], ['New Zealand-Portugal'], ['Australia-Germany'], ['Cameroon-Australia'], ['Germany-Chile'], ['Germany-Cameroon'], ['Chile-Australia']]


,+/-_x,+/-_y,Previous Points_x,Previous Points_y,Total Points_x,Total Points_y
0,-2,-2,561,1259,560.85,1266.76
1,-1,-1,1076,301,1049.98,343.62
2,-1,-1,1076,561,1049.98,560.85
3,17,17,301,1259,343.62,1266.76
4,2,2,661,1464,681.26,1511.44
5,1,1,779,661,811.05,681.26
6,0,0,1464,1411,1511.44,1422.14
7,0,0,1464,779,1511.44,811.05
8,0,0,1411,661,1422.14,681.26


## Suport Vector Machine

In [18]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

grid_Params={'C':[0.01, 0.1, 1, 10, 100], 'gamma':[0.01, 0.1, 1, 10, 100]}

grid1=GridSearchCV(SVC(kernel='rbf'), param_grid=grid_Params, scoring='precision_micro')
grid1.fit(X_train, y_train)

print('Best params SVC: ', grid1.best_params_)
print('Best score SVC: ', grid1.best_score_)

dtf_res=pd.DataFrame(grid1.cv_results_)
print(dtf_res)

Best params SVC:  {'C': 1, 'gamma': 0.1}
Best score SVC:  0.507389162562
    mean_fit_time  mean_score_time  mean_test_score  mean_train_score param_C  \
0        0.010737         0.003676         0.502463          0.502469    0.01   
1        0.008093         0.002622         0.502463          0.502469    0.01   
2        0.007453         0.002485         0.502463          0.502469    0.01   
3        0.007781         0.002583         0.502463          0.502469    0.01   
4        0.007531         0.002505         0.502463          0.502469    0.01   
5        0.007813         0.003336         0.502463          0.502469     0.1   
6        0.007763         0.002581         0.502463          0.502469     0.1   
7        0.007808         0.002538         0.502463          0.502469     0.1   
8        0.007591         0.002564         0.502463          0.502469     0.1   
9        0.007634         0.002537         0.502463          0.502469     0.1   
10       0.007913         0.002575  

In [19]:
lr=list(zip(for_result_lst, grid1.predict(pd_query)))
for i in lr:
    print(i)

(['Russia-Portugal'], 1)
(['Mexico-New Zealand'], 1)
(['Mexico-Russia'], 1)
(['New Zealand-Portugal'], 1)
(['Australia-Germany'], 1)
(['Cameroon-Australia'], 1)
(['Germany-Chile'], 1)
(['Germany-Cameroon'], 1)
(['Chile-Australia'], 1)


## Logistic Regression

In [21]:
from sklearn.linear_model import LogisticRegression
grid_ParamsLG={'C':[0.01, 0.1, 1, 10, 100]}

grid4=GridSearchCV(LogisticRegression(), param_grid=grid_ParamsLG, scoring='precision_micro')
grid4.fit(X_train, y_train)

print('Best params (max. accurancy): ', grid4.best_params_)
print('Best score (accurancy): ', grid4.best_score_)

dtf_res=pd.DataFrame(grid4.cv_results_)
print(dtf_res)

Best params (max. accurancy):  {'C': 1}
Best score (accurancy):  0.679802955665
   mean_fit_time  mean_score_time  mean_test_score  mean_train_score param_C  \
0       0.006273         0.003627         0.669951          0.692121    0.01   
1       0.009819         0.004126         0.669951          0.694590     0.1   
2       0.006643         0.001253         0.679803          0.699492       1   
3       0.006298         0.001140         0.674877          0.706881      10   
4       0.006943         0.001191         0.669951          0.704430     100   

        params  rank_test_score  split0_test_score  split0_train_score  \
0  {'C': 0.01}                3           0.735294            0.666667   
1   {'C': 0.1}                3           0.735294            0.674074   
2     {'C': 1}                1           0.720588            0.674074   
3    {'C': 10}                2           0.720588            0.688889   
4   {'C': 100}                3           0.720588            0.68888

In [22]:
lr=list(zip(for_result_lst, grid4.predict(pd_query)))
for i in lr:
    print(i)

(['Russia-Portugal'], 1)
(['Mexico-New Zealand'], -1)
(['Mexico-Russia'], -1)
(['New Zealand-Portugal'], 1)
(['Australia-Germany'], 1)
(['Cameroon-Australia'], -1)
(['Germany-Chile'], -1)
(['Germany-Cameroon'], -1)
(['Chile-Australia'], -1)


## Decision Tree

In [44]:
from sklearn import tree

grid_ParamsDT={'max_depth':[1, 3, 5, 10, 15, 30, 50, 100]}

grid2=GridSearchCV(tree.DecisionTreeClassifier(), param_grid=grid_ParamsDT, scoring='accuracy')
grid2.fit(X_train, y_train)

print('Best params Tree (max. accurancy): ', grid2.best_params_)
print('Best score Tree (accurancy): ', grid2.best_score_)

dtf_res=pd.DataFrame(grid2.cv_results_)
print(dtf_res)

Best params Tree (max. accurancy):  {'max_depth': 3}
Best score Tree (accurancy):  0.620689655172
   mean_fit_time  mean_score_time  mean_test_score  mean_train_score  \
0       0.003494         0.000635         0.561576          0.598475   
1       0.005093         0.000628         0.620690          0.721641   
2       0.006272         0.000863         0.527094          0.812745   
3       0.004217         0.000809         0.527094          0.982716   
4       0.004187         0.000829         0.546798          0.997531   
5       0.004157         0.000797         0.551724          0.997531   
6       0.003973         0.000772         0.536946          0.997531   
7       0.003935         0.000803         0.571429          0.997531   

  param_max_depth              params  rank_test_score  split0_test_score  \
0               1    {'max_depth': 1}                3           0.529412   
1               3    {'max_depth': 3}                1           0.617647   
2               5    {

In [45]:
lr=list(zip(for_result_lst, grid2.predict(pd_query)))
for i in lr:
    print(i)

(['Russia-Portugal'], -1)
(['Mexico-New Zealand'], -1)
(['Mexico-Russia'], -1)
(['New Zealand-Portugal'], -1)
(['Australia-Germany'], -1)
(['Cameroon-Australia'], -1)
(['Germany-Chile'], -1)
(['Germany-Cameroon'], -1)
(['Chile-Australia'], -1)


## KNeighbors Classifier

In [46]:
from sklearn.neighbors import KNeighborsClassifier 

grid_ParamsKNN={'n_neighbors':[1, 3, 5, 15, 20, 30, 35,  50, 70, 100]}

grid3=GridSearchCV(KNeighborsClassifier(), param_grid=grid_ParamsKNN, scoring='accuracy')
grid3.fit(X_train, y_train)

print('Best params Neighbors: ', grid3.best_params_)
print('Best score Neighbors: ', grid3.best_score_)

dtf_res=pd.DataFrame(grid3.cv_results_)
print(dtf_res)

#'precsion_micro' is not a valid scoring value. Valid options are 
#['accuracy', 'adjusted_rand_score', 'average_precision', 'f1', 'f1_macro', 
# 'f1_micro', 'f1_samples', 'f1_weighted', 'neg_log_loss', 'neg_mean_absolute_error', 
# 'neg_mean_squared_error', 'neg_median_absolute_error', 'precision', 'precision_macro', 
# 'precision_micro', 'precision_samples', 'precision_weighted', 'r2', 'recall', 'recall_macro', 
# 'recall_micro', 'recall_samples', 'recall_weighted', 'roc_auc']

Best params Neighbors:  {'n_neighbors': 30}
Best score Neighbors:  0.699507389163
   mean_fit_time  mean_score_time  mean_test_score  mean_train_score  \
0       0.004447         0.001541         0.571429          0.997531   
1       0.003654         0.001534         0.591133          0.758678   
2       0.005916         0.002833         0.620690          0.721714   
3       0.003978         0.002118         0.640394          0.704484   
4       0.003707         0.002097         0.674877          0.694553   
5       0.003673         0.002191         0.699507          0.692121   
6       0.003630         0.002286         0.674877          0.689706   
7       0.003666         0.002450         0.674877          0.674927   
8       0.003508         0.002543         0.665025          0.677397   
9       0.003504         0.002553         0.610837          0.605955   

  param_n_neighbors                params  rank_test_score  split0_test_score  \
0                 1    {'n_neighbors': 1}   

In [47]:
lr=list(zip(for_result_lst, grid3.predict(pd_query)))
for i in lr:
    print(i)

print(grid3.score(X_test, y_test))

(['Russia-Portugal'], -1)
(['Mexico-New Zealand'], -1)
(['Mexico-Russia'], -1)
(['New Zealand-Portugal'], -1)
(['Australia-Germany'], -1)
(['Cameroon-Australia'], -1)
(['Germany-Chile'], -1)
(['Germany-Cameroon'], -1)
(['Chile-Australia'], -1)
0.558823529412
